**WEEK 6**

In [ ]:
!pip install update transformers
!pip install datasets


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
sys.path.append('/Users/gonzalovillalbamartinez/opt/miniconda3/lib/python3.9/site-packages')

In [ ]:
from datasets import load_dataset
from datasets import load_metric
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import AutoConfig
from functools import partial
import torch
import random
import numpy as np
from tqdm import tqdm
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.optim.lr_scheduler import LambdaLR
from torch import nn
from collections import defaultdict, OrderedDict
MODEL_NAME = 'xlm-roberta-base'
# MODEL_NAME = 'bert-base-uncased'

In [ ]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

enforce_reproducibility()

In [ ]:
""" Official evaluation script for v1.1 of the SQuAD dataset. """
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate_squad(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

def compute_squad(predictions, references):
  pred_dict = {prediction["id"]: prediction["prediction_text"] for prediction in predictions}
  dataset = [
      {
          "paragraphs": [
              {
                  "qas": [
                      {
                          "answers": [{"text": answer_text} for answer_text in ref["answers"]["text"]],
                          "id": ref["id"],
                      }
                      for ref in references
                  ]
              }
          ]
      }
  ]
  score = evaluate_squad(dataset=dataset, predictions=pred_dict)
  return score

In [ ]:
# this is also equivalent to those 2 lines. I recommend going with that, unless you want more control over your code
from datasets import load_metric
compute_squad = load_metric("squad_v2")

In [ ]:
# corpus_questions_eng
# corpus_context_eng

# corpus_questions_eng_not_tokenized
# corpus_context_eng_not_tokenized


['When was quantum field theory developed ?',
 'Who was the first Nobel prize winner for Literature ?',
 'When is the dialectical method used ?',
 'Who invented Hangul ?',
 'What do Grasshoppers eat ?',
 'How large is the Kerman province of southeastern Iran ?',
 'When was Guitar Hero Live first released ?',
 'When were bluebonnets named the state flower of Texas ?',
 'Who created the series Clannad ?',
 'When was the USS Taylor built ?',
 "What 's the difference between man-slaughter and homicide ?",
 'When did DC comics first introduce the Guardians of the Universe ?',
 'What is the largest parrot in New Zealand ?',
 "When was the first Assassin 's Creed released ?",
 "What percentage of the Earth 's atmosphere is oxygen ?",
 'When was the X-Men cartoon television series first aired ?',
 'When was the Battle of Suoi Chau Pha fought ?',
 'How old was Cho Namchul when he died ?',
 "What is the Navy 's military mascot ?",
 'How many permanent members of the United Nations Security Counc

In [ ]:
tk = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading: 100%|██████████| 615/615 [00:00<00:00, 277kB/s]
Downloading: 100%|██████████| 5.07M/5.07M [00:01<00:00, 4.83MB/s]
Downloading: 100%|██████████| 9.10M/9.10M [00:01<00:00, 6.92MB/s]


In [ ]:
# batch = tk.encode_plus(
#     corpus_questions_eng_not_tokenized[70],
#     corpus_context_eng_not_tokenized[70],
#     padding='max_length',
#     truncation='only_second',
#     stride=128,
#     return_overflowing_tokens=True,
#     return_offsets_mapping=True
# )

# # Get a list which maps the input features index to their original index in the 
# # samples list (for split inputs). E.g. if our batch size is 4 and the second sample
# # is split into 3 inputs because it is very large, sample_mapping would look like
# # [0, 1, 1, 1, 2, 3]
# sample_mapping = batch.pop('overflow_to_sample_mapping')
# # Get all of the character offsets for each token
# offset_mapping = batch.pop('offset_mapping')

In [ ]:
# list(batch.keys())

['input_ids', 'attention_mask']

In [ ]:
# sample_mapping

[0]

In [ ]:
# len(offset_mapping)

1

In [ ]:
# offset_mapping[0][:50]

[(0, 0),
 (0, 3),
 (4, 7),
 (8, 10),
 (11, 14),
 (14, 17),
 (18, 21),
 (21, 24),
 (25, 29),
 (30, 36),
 (37, 38),
 (0, 0),
 (0, 0),
 (0, 3),
 (3, 6),
 (7, 10),
 (10, 13),
 (14, 15),
 (16, 17),
 (17, 19),
 (19, 20),
 (20, 22),
 (22, 23),
 (24, 25),
 (26, 32),
 (33, 34),
 (35, 36),
 (36, 39),
 (39, 40),
 (41, 42),
 (43, 44),
 (43, 44),
 (45, 50),
 (50, 52),
 (53, 55),
 (55, 59),
 (59, 60),
 (60, 64),
 (65, 67),
 (68, 70),
 (71, 72),
 (71, 72),
 (73, 75),
 (76, 77),
 (78, 83),
 (84, 88),
 (89, 90),
 (91, 95),
 (96, 97),
 (98, 99)]

In [ ]:
def get_train_features(tk, samples):
  '''
  Tokenizes all of the text in the given samples, splittling inputs that are too long for our model
  across multiple features. Finds the token offsets of the answers, which serve as the labels for
  our inputs.
  '''
  batch = tk.batch_encode_plus(
        [[q,c] for q,c in zip(samples['question'], samples['context'])], 
        padding='max_length', 
        truncation='only_second',
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True
    )

  # Get a list which maps the input features index to their original index in the 
  # samples list (for split inputs). E.g. if our batch size is 4 and the second sample
  # is split into 3 inputs because it is very large, sample_mapping would look like
  # [0, 1, 1, 1, 2, 3]
  sample_mapping = batch.pop('overflow_to_sample_mapping')
  # Get all of the character offsets for each token
  offset_mapping = batch.pop('offset_mapping')

  # Store the start and end tokens
  batch['start_tokens'] = []
  batch['end_tokens'] = []

  # Iterate through all of the offsets
  for i, offsets in enumerate(offset_mapping):
    # Get the right sample by mapping it to its original index
    sample_idx = sample_mapping[i]
    # Get the sequence IDs to know where context starts so we can ignore question tokens
    sequence_ids = batch.sequence_ids(i)

    # Get the start and end character positions of the answer
    ans = samples['answers'][sample_idx]
    start_char = ans['answer_start'][0]
    end_char = start_char + len(ans['text'][0])
    # while end_char > 0 and (end_char >= len(samples['context'][sample_idx]) or samples['context'][sample_idx][end_char] == ' '):
    #   end_char -= 1

    # Start from the first token in the context, which can be found by going to the 
    # first token where sequence_ids is 1
    start_token = 0
    while sequence_ids[start_token] != 1:
      start_token += 1

    end_token = len(offsets) - 1
    while sequence_ids[end_token] != 1:
      end_token -= 1

    # By default set it to the CLS token if the answer isn't in this input
    if start_char < offsets[start_token][0] or end_char > offsets[end_token][1]:
      start_token = 0
      end_token = 0
    # Otherwise find the correct token indices
    else:
      # Advance the start token index until we have passed the start character index 
      while start_token < len(offsets) and offsets[start_token][0] <= start_char:
        start_token += 1
      start_token -= 1
      
      # Decrease the end token index until we have passed the end character index
      while end_token >= 0 and offsets[end_token][1] >= end_char:
        end_token -= 1
      end_token += 1

    batch['start_tokens'].append(start_token)
    batch['end_tokens'].append(end_token)

  #batch['start_tokens'] = np.array(batch['start_tokens'])
  #batch['end_tokens'] = np.array(batch['end_tokens'])

  return batch

def collate_fn(inputs):
  '''
  Defines how to combine different samples in a batch
  '''
  input_ids = torch.tensor([i['input_ids'] for i in inputs])
  attention_mask = torch.tensor([i['attention_mask'] for i in inputs])
  start_tokens = torch.tensor([i['start_tokens'] for i in inputs])
  end_tokens = torch.tensor([i['end_tokens'] for i in inputs])

  # Truncate to max length
  max_len = max(attention_mask.sum(-1))
  input_ids = input_ids[:,:max_len]
  attention_mask = attention_mask[:,:max_len]
  
  return {'input_ids': input_ids, 'attention_mask': attention_mask, 'start_tokens': start_tokens, 'end_tokens': end_tokens}

In [ ]:
# samples = {'question': corpus_questions_eng_not_tokenized, 'context': corpus_context_eng_not_tokenized}

# tokenized_dataset = get_train_features(tk, samples)

# tokenized_dataset = mlqa['test'].map(partial(get_train_features, tk), batched=True, remove_columns=mlqa['test'].column_names)


## Get data well-formated from Week 5's IOB


In [ ]:
english_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
#train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=8)

## Check what dataset to use here. 'training_data_eng' is a list and thus can't access training_data_eng['question'] or training_data_eng['context']

# tokenized_dataset = get_train_features(tk, samples)


tokenized_dataset = training_data_finn.map(partial(get_train_features, tk))

english_model.to('cpu')

In [ ]:
samples = random.sample(list(range(len(tokenized_dataset))), 4000)
tokenized_dataset = tokenized_dataset.select(samples)
train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)

{'question': ['When was quantum field theory developed ?',
  'Who was the first Nobel prize winner for Literature ?',
  'When is the dialectical method used ?',
  'Who invented Hangul ?',
  'What do Grasshoppers eat ?',
  'How large is the Kerman province of southeastern Iran ?',
  'When was Guitar Hero Live first released ?',
  'When were bluebonnets named the state flower of Texas ?',
  'Who created the series Clannad ?',
  'When was the USS Taylor built ?',
  "What 's the difference between man-slaughter and homicide ?",
  'When did DC comics first introduce the Guardians of the Universe ?',
  'What is the largest parrot in New Zealand ?',
  "When was the first Assassin 's Creed released ?",
  "What percentage of the Earth 's atmosphere is oxygen ?",
  'When was the X-Men cartoon television series first aired ?',
  'When was the Battle of Suoi Chau Pha fought ?',
  'How old was Cho Namchul when he died ?',
  "What is the Navy 's military mascot ?",
  'How many permanent members of t

In [ ]:
# Create the optimizer
lr=2e-5
n_epochs = 3
weight_decay = 0.01
warmup_steps = 200

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in english_model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': weight_decay},
    {'params': [p for n, p in english_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
# optimizer = Adam(optimizer_grouped_parameters, lr=1e-3)
# scheduler = None
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    warmup_steps,
    n_epochs * len(train_dl)
)

losses = train(
    english_model, 
    train_dl,
    optimizer, 
    scheduler,
    n_epochs, 
    device
)

In [ ]:
logits = predict(english_model, valid_dl)

In [ ]:

predictions = post_process_predictions(validation_data_eng, valid_dl, logits)
formatted_predictions = [{'id': k, 'prediction_text': v} for k,v in predictions.items()]
gold = [{'id': example['id'], 'answers': example['answers']} for example in valid_dl]
compute_squad(references=gold, predictions=formatted_predictions)

In [ ]:
training_data_eng[6]

'When was Guitar Hero Live first released ?\nGuitar Hero Live is a music video game developed by FreeStyleGames and published by Activision . It was released for PlayStation 3 , PlayStation 4 , Wii U , Xbox 360 , and Xbox One in October 2015 and to iOS devices including Apple TV in November 2015 . As with previous games in the series , the goal is to use a special guitar controller to match fret patterns displayed on a scrolling note pattern on screen in time with the music .'

Trained in Finnish

In [ ]:
## Get tokenized Finnish datased

finnish_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
#train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=8)

In [ ]:
## Check what dataset to use here. 'training_data_finn' is a list and thus can't access training_data_finn['question'] or training_data_fin['context']

# tokenized_dataset = get_train_features(tk, samples)


tokenized_dataset = training_data_finn.map(partial(get_train_features, tk))

finnish_model.to('cpu')

finnish_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
#train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=8)

In [ ]:
samples = random.sample(list(range(len(tokenized_dataset))), 4000)
tokenized_dataset = tokenized_dataset.select(samples)
train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)

In [ ]:
# Create the optimizer
lr=2e-5
n_epochs = 3
weight_decay = 0.01
warmup_steps = 200

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in english_model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': weight_decay},
    {'params': [p for n, p in english_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
# optimizer = Adam(optimizer_grouped_parameters, lr=1e-3)
# scheduler = None
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    warmup_steps,
    n_epochs * len(train_dl)
)

## Check losses with finnish model
losses = train(
    finnish_model, 
    train_dl,
    optimizer, 
    scheduler,
    n_epochs, 
    device
)

In [ ]:
logits = predict(finnish_model, valid_dl)

In [ ]:
predictions = post_process_predictions(validation_data_finn, valid_dl, logits)
formatted_predictions = [{'id': k, 'prediction_text': v} for k,v in predictions.items()]
gold = [{'id': example['id'], 'answers': example['answers']} for example in valid_dl]
compute_squad(references=gold, predictions=formatted_predictions)

Trained in Japanese

In [ ]:
## Get tokenized Japanese datased

japanese_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
#train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=8)

In [ ]:
## Check what dataset to use here. 'training_data_jap' is a list and thus can't access training_data_jap['question'] or training_data_jap['context']

# tokenized_dataset = get_train_features(tk, samples)


tokenized_dataset = training_data_jap.map(partial(get_train_features, tk))

japanese_model.to('cpu')

japanese_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
#train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=8)

In [ ]:
samples = random.sample(list(range(len(tokenized_dataset))), 4000)
tokenized_dataset = tokenized_dataset.select(samples)
train_dl = DataLoader(tokenized_dataset, collate_fn=collate_fn, shuffle=True, batch_size=4)

In [ ]:
# Create the optimizer
lr=2e-5
n_epochs = 3
weight_decay = 0.01
warmup_steps = 200

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in english_model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': weight_decay},
    {'params': [p for n, p in english_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
# optimizer = Adam(optimizer_grouped_parameters, lr=1e-3)
# scheduler = None
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    warmup_steps,
    n_epochs * len(train_dl)
)

## Check losses with finnish model
losses = train(
    japanese_model, 
    train_dl,
    optimizer, 
    scheduler,
    n_epochs, 
    device
)

In [ ]:
logits = predict(japanese_model, valid_dl)

In [ ]:
predictions = post_process_predictions(validation_data_jap, valid_dl, logits)
formatted_predictions = [{'id': k, 'prediction_text': v} for k,v in predictions.items()]
gold = [{'id': example['id'], 'answers': example['answers']} for example in valid_dl]
compute_squad(references=gold, predictions=formatted_predictions)

Try English and Japanese to Finnish.
Try English and Finnish to Japanese.
Compare

In [ ]:

samples = data_set['val'][70]
batch = tk.encode_plus(
        samples['question'], 
        samples['context'], 
        padding='max_length', 
        truncation='only_second',
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True
    )

# Get a list which maps the input features index to their original index in the 
# samples list (for split inputs). E.g. if our batch size is 4 and the second sample
# is split into 3 inputs because it is very large, sample_mapping would look like
# [0, 1, 1, 1, 2, 3]
sample_mapping = batch.pop('overflow_to_sample_mapping')
# Get all of the character offsets for each token
offset_mapping = batch.pop('offset_mapping')